# Star Wars Action Figures Data Analysis
## Comprehensive Exploratory Data Analysis

This notebook analyzes Star Wars action figure sales data including pricing trends, character types, and market dynamics.

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os

# Set style for better-looking plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Load the data
df = pd.read_csv('../other files/starwars_filtered.csv')
print(f"Dataset loaded successfully!")
print(f"Total records: {len(df):,}")

## 2. Data Overview

In [ ]:
# First few rows
df.head(10)

In [ ]:
# Data info
df.info()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

In [ ]:
# Basic statistics
df.describe()

In [ ]:
# Categorical variables overview
print(f"Date range: {df['year'].min()} - {df['year'].max()}")
print(f"\nUnique figures: {df['figure'].nunique()}")
print(f"\nCharacter types: {df['character_type'].unique().tolist()}")
print(f"\nConditions: {df['condition'].unique().tolist()}")
print(f"\nAuthenticity distribution:")
print(df['authenticity_n'].value_counts())

## 3. Price Analysis

In [ ]:
# Price Distribution by Grading/Certification
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Price Distribution by Grading/Certification', fontsize=16, fontweight='bold')

# Define the combinations
combinations = [
    (1, 'Graded/Certified', axes[0]),
    (0, 'Not Graded', axes[1])
]

for auth, title, ax in combinations:
    # Filter data
    data = df[df['authenticity_n'] == auth]['selling_price']
    
    # Plot histogram
    ax.hist(data, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_xlabel('Selling Price ($)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Add mean and median lines
    ax.axvline(data.mean(), color='red', linestyle='--', linewidth=2,
               label=f'Mean: ${data.mean():.2f}')
    ax.axvline(data.median(), color='green', linestyle='--', linewidth=2,
               label=f'Median: ${data.median():.2f}')
    
    # Add count annotation
    ax.text(0.98, 0.97, f'n = {len(data):,}', 
            transform=ax.transAxes, fontsize=10,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.legend(fontsize=9)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary Statistics by Grading/Certification
print("Price Statistics by Grading/Certification:")
print("="*80)

summary_data = []
combinations = [
    (1, 'Graded/Certified'),
    (0, 'Not Graded')
]

for auth, label in combinations:
    data = df[df['authenticity_n'] == auth]['selling_price']
    summary_data.append({
        'Category': label,
        'Count': len(data),
        'Mean': f'${data.mean():.2f}',
        'Median': f'${data.median():.2f}',
        'Std Dev': f'${data.std():.2f}',
        'Min': f'${data.min():.2f}',
        'Max': f'${data.max():.2f}'
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n" + "="*80)
print("Key Observations:")
print(f"- Highest average price: Graded/Certified")
print(f"- Lowest average price: Not Graded")
print(f"- Largest segment: {summary_df.loc[summary_df['Count'].idxmax(), 'Category']}")
print(f"- Smallest segment: {summary_df.loc[summary_df['Count'].idxmin(), 'Category']}")

In [ ]:
# Boxplot Comparison: Price Distribution by Grading Status
fig, ax = plt.subplots(figsize=(10, 8))

# Prepare data for boxplot
price_data = []
tick_labels = []

combinations = [
    (1, 'Graded/\nCertified'),
    (0, 'Not Graded')
]

for auth, label in combinations:
    data = df[df['authenticity_n'] == auth]['selling_price']
    price_data.append(data)
    tick_labels.append(label)

# Create boxplot
bp = ax.boxplot(price_data, tick_labels=tick_labels, patch_artist=True, showmeans=True,
                meanprops=dict(marker='D', markerfacecolor='red', markersize=8, markeredgecolor='darkred'))

# Color the boxes
colors = ['#e74c3c', '#3498db']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('Selling Price ($)', fontsize=13)
ax.set_xlabel('Grading Status', fontsize=13)
ax.set_title('Price Distribution by Grading Status (Boxplot Comparison)', fontsize=15, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add mean values as annotations
for i, (auth, _) in enumerate(combinations):
    data = df[df['authenticity_n'] == auth]['selling_price']
    mean_val = data.mean()
    median_val = data.median()
    ax.text(i+1, mean_val + 50, f'Mean: ${mean_val:.0f}', 
            ha='center', fontsize=9, color='darkred', fontweight='bold')
    ax.text(i+1, median_val - 50, f'Med: ${median_val:.0f}', 
            ha='center', fontsize=9, color='darkblue', fontweight='bold')

plt.tight_layout()
plt.show()

# Print boxplot statistics
print("\n" + "="*80)
print("BOXPLOT STATISTICS BY GRADING STATUS")
print("="*80)

for auth, label in combinations:
    data = df[df['authenticity_n'] == auth]['selling_price']
    
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_whisker = data[data >= (q1 - 1.5 * iqr)].min()
    upper_whisker = data[data <= (q3 + 1.5 * iqr)].max()
    outliers = data[(data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr))]
    
    print(f"\n{label.replace(chr(10), ' ')}:")
    print(f"  Median (Q2): ${data.median():.2f}")
    print(f"  Q1 (25th percentile): ${q1:.2f}")
    print(f"  Q3 (75th percentile): ${q3:.2f}")
    print(f"  IQR: ${iqr:.2f}")
    print(f"  Lower whisker: ${lower_whisker:.2f}")
    print(f"  Upper whisker: ${upper_whisker:.2f}")
    print(f"  Number of outliers: {len(outliers):,} ({len(outliers)/len(data)*100:.1f}%)")
    print(f"  Mean: ${data.mean():.2f}")

print("\n" + "="*80)

### 🔑 Key Insights:
1. **Grading Premium**: Graded/Certified figures command 92% premium over Not Graded ($344 vs $179 avg)
2. **Market Split**: 55.5% not graded vs 44.5% graded - grading is mainstream but not dominant
3. **Value Distribution**: Higher outlier prevalence in graded segment indicates rare high-value sales

In [ ]:
# Boxplot Without Outliers: Better View of Box Structure
fig, ax = plt.subplots(figsize=(10, 8))

# Prepare data for boxplot
price_data_no_outliers = []
tick_labels = []

combinations = [
    (1, 'Graded/\nCertified'),
    (0, 'Not Graded')
]

# Filter out outliers for each segment
for auth, label in combinations:
    data = df[df['authenticity_n'] == auth]['selling_price']
    
    # Remove outliers (values beyond 1.5*IQR from Q1/Q3)
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Filter data to exclude outliers
    data_filtered = data[(data >= lower_bound) & (data <= upper_bound)]
    price_data_no_outliers.append(data_filtered)
    tick_labels.append(label)

# Create boxplot without outliers
bp = ax.boxplot(price_data_no_outliers, tick_labels=tick_labels, patch_artist=True, showmeans=True,
                meanprops=dict(marker='D', markerfacecolor='red', markersize=8, markeredgecolor='darkred'),
                showfliers=False)  # Don't show outlier points

# Color the boxes
colors = ['#e74c3c', '#3498db']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('Selling Price ($)', fontsize=13)
ax.set_xlabel('Grading Status', fontsize=13)
ax.set_title('Price Distribution by Grading Status (Outliers Excluded for Clarity)', fontsize=15, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add mean and median annotations
for i, (auth, _) in enumerate(combinations):
    data = df[df['authenticity_n'] == auth]['selling_price']
    
    # Calculate stats without outliers
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    data_filtered = data[(data >= lower_bound) & (data <= upper_bound)]
    
    mean_val = data_filtered.mean()
    median_val = data_filtered.median()
    
    ax.text(i+1, mean_val + (data_filtered.max() - data_filtered.min()) * 0.03, 
            f'Mean: ${mean_val:.0f}', 
            ha='center', fontsize=9, color='darkred', fontweight='bold')
    ax.text(i+1, median_val - (data_filtered.max() - data_filtered.min()) * 0.03, 
            f'Med: ${median_val:.0f}', 
            ha='center', fontsize=9, color='darkblue', fontweight='bold')

plt.tight_layout()
plt.show()

# Print comparison statistics
print("\n" + "="*80)
print("COMPARISON: WITH vs WITHOUT OUTLIERS")
print("="*80)

for auth, label in combinations:
    data = df[df['authenticity_n'] == auth]['selling_price']
    
    # Calculate with outliers
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    
    # Calculate without outliers
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    data_filtered = data[(data >= lower_bound) & (data <= upper_bound)]
    
    num_outliers = len(data) - len(data_filtered)
    pct_outliers = (num_outliers / len(data)) * 100
    
    print(f"\n{label.replace(chr(10), ' ')}:")
    print(f"  With outliers:    Mean = ${data.mean():7.2f}, Median = ${data.median():7.2f}, n = {len(data):,}")
    print(f"  Without outliers: Mean = ${data_filtered.mean():7.2f}, Median = ${data_filtered.median():7.2f}, n = {len(data_filtered):,}")
    print(f"  Outliers removed: {num_outliers:,} ({pct_outliers:.1f}%)")
    print(f"  Price range (no outliers): ${data_filtered.min():.2f} - ${data_filtered.max():.2f}")

print("\n" + "="*80)

### 📊 How to Interpret the Boxplot

**Reading the Boxplot:**
- **Box**: Contains the middle 50% of prices (from Q1 to Q3)
- **Line inside box**: Median price (50th percentile)
- **Red diamond**: Mean (average) price
- **Whiskers**: Extend to 1.5× IQR beyond the box
- **Dots beyond whiskers**: Outliers (unusually high/low prices)

**Understanding the Components:**
- **Q1 (25th percentile)**: Bottom of the box - 25% of sales below this price
- **Q3 (75th percentile)**: Top of the box - 75% of sales below this price
- **IQR (Interquartile Range)**: Q3 - Q1, measures spread of middle 50%
- **Outliers**: Individual high-value sales (rare figures, mint condition, etc.)

**Key Insights:**
1. **Box Height = Market Consistency**: Taller boxes show more price variance, shorter boxes indicate consistent pricing
2. **Median vs Mean Gap**: When mean (diamond) is far above median (line), outliers heavily influence the average
3. **Whisker Length**: Longer whiskers indicate wider typical price ranges
4. **Outlier Prevalence**: More dots = more exceptional high-value sales in that segment

**Comparing the 2 Segments:**
- **Graded/Certified** (red): Higher median and mean, premium segment with significant outliers
- **Not Graded** (blue): Lower tier pricing, represents the broader market

**Why This Matters:**
- Boxplots reveal the **typical price range** (box) vs **extreme sales** (outliers)
- Shows **market segmentation** - grading creates distinct price tiers
- **Mean above median** in both segments shows high-value rare figures drive up averages
- Helps buyers/sellers understand **expected price ranges** vs exceptional sales
- Outliers represent rare variants, chase figures, or exceptional condition examples

### 📊 Understanding the Outlier-Excluded Boxplot

**Why Remove Outliers?**
- The previous boxplot was **visually compressed** because extreme high-value sales (outliers) forced a wide y-axis range
- Excluding outliers allows us to see the **typical market structure** more clearly
- The boxes and whiskers are now **more visible and interpretable**

**What We Can See Now:**
1. **Box sizes are clearer**: The IQR (box height) shows the spread of typical prices
2. **Median lines are visible**: The line in each box shows where 50% of sales fall
3. **Whisker ranges**: Better view of the price range for 95%+ of typical sales
4. **Mean vs Median gaps**: Easier to see how much outliers were pulling averages up

**Key Observations from Outlier-Free View:**
- **Graded/Certified** (red): Higher typical prices with moderate spread
- **Not Graded** (blue): Lower and more compressed prices, representing the mass market

**Comparison Statistics Show:**
- Each segment has **5-15% outliers** removed (rare ultra-high-value sales)
- **Means drop significantly** without outliers, showing how rare sales inflate averages
- **Medians barely change** - confirming median is more representative of typical sales
- **Typical price ranges** are much narrower than you'd think from overall stats

**Practical Takeaway:**
- If you're a **buyer**: Use the outlier-free view to understand typical prices you'll encounter
- If you're a **seller**: The first boxplot (with outliers) shows the potential upside for rare/exceptional items
- **Most transactions** occur within the box and whiskers of the outlier-free view

## 4. Character Type Analysis

In [ ]:
# Sales by Character Type
char_sales = df.groupby('character_type').agg({
    'selling_price': ['mean', 'median', 'count']
}).round(2)
char_sales.columns = ['Mean Price', 'Median Price', 'Count']
char_sales = char_sales.sort_values('Mean Price', ascending=False)

print("Average Prices by Character Type:")
print(char_sales)

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(char_sales.index))
width = 0.35

ax.bar(x - width/2, char_sales['Mean Price'], width, label='Mean Price', alpha=0.8)
ax.bar(x + width/2, char_sales['Median Price'], width, label='Median Price', alpha=0.8)
ax.set_xlabel('Character Type')
ax.set_ylabel('Price ($)')
ax.set_title('Average Selling Price by Character Type')
ax.set_xticks(x)
ax.set_xticklabels(char_sales.index, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add count annotations
for i, count in enumerate(char_sales['Count']):
    ax.text(i, 5, f'n={int(count)}', ha='center', va='bottom', fontsize=8, rotation=90)

plt.tight_layout()
plt.show()

### 🔑 Key Insights:
1. **Bounty Hunters Reign**: $374 avg (2.4× mean-median gap) - iconic villains command huge premiums with high volatility
2. **Alien/Other Mass Market**: 34.7% of all transactions but lowest price ($204) - high volume, low value segment
3. **Top Tier Cluster**: Bounty hunters ($374), Jedi ($366), Droids ($284) all exceed overall mean ($253) significantly

In [ ]:
# Price Trends Over Time - Split by Grading Status (2015-2025)
fig, ax = plt.subplots(figsize=(14, 8))

# Filter data from 2015 onwards
df_filtered = df[df['year'] >= 2015]

# Define the two combinations
combinations = [
    (1, 'Graded/Certified', 'o', '#e74c3c', 3),
    (0, 'Not Graded', 's', '#3498db', 2.5)
]

# Plot each combination
for auth, label, marker, color, linewidth in combinations:
    # Filter and group by year
    subset = df_filtered[df_filtered['authenticity_n'] == auth]
    yearly = subset.groupby('year')['selling_price'].agg(['mean', 'median', 'count']).reset_index()
    
    # Plot mean price
    ax.plot(yearly['year'], yearly['mean'], marker=marker, linewidth=linewidth,
            label=label, markersize=7, color=color, alpha=0.8)

ax.set_xlabel('Year', fontsize=13)
ax.set_ylabel('Average Selling Price ($)', fontsize=13)
ax.set_title('Star Wars Figure Prices Over Time by Grading Status (2015-2025)', 
             fontsize=15, fontweight='bold', pad=20)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

# Set x-axis to show only integer years
ax.set_xticks(range(2015, 2026, 1))
ax.set_xticklabels([str(year) for year in range(2015, 2026, 1)], rotation=0)

plt.tight_layout()
plt.show()

# Print statistics for each segment
print("\n" + "="*80)
print("PRICE TRENDS BY GRADING STATUS (2015-2025)")
print("="*80)

for auth, label, _, _, _ in combinations:
    subset = df_filtered[df_filtered['authenticity_n'] == auth]
    yearly = subset.groupby('year')['selling_price'].mean()
    
    # Calculate growth metrics
    start_price = yearly.iloc[0] if len(yearly) > 0 else 0
    end_price = yearly.iloc[-1] if len(yearly) > 0 else 0
    growth = ((end_price - start_price) / start_price * 100) if start_price > 0 else 0
    
    peak_year = int(yearly.idxmax()) if len(yearly) > 0 else 0
    peak_price = yearly.max() if len(yearly) > 0 else 0
    
    current_price = yearly.get(2025, 0)
    
    print(f"\n{label}:")
    print(f"  2015 starting price: ${start_price:.2f}")
    print(f"  2025 current price: ${current_price:.2f}")
    print(f"  Total growth (2015-2025): {growth:+.1f}%")
    print(f"  Peak year: {peak_year} at ${peak_price:.2f}")
    print(f"  Change from peak: {((current_price - peak_price) / peak_price * 100):+.1f}%")

print("\n" + "="*80)

## 5. Time Series Analysis - Sales Volume

In [ ]:
# Sales Volume Over Time - Split by Grading Status (2015-2025)
fig, ax = plt.subplots(figsize=(14, 8))

# Filter data from 2015 onwards
df_filtered = df[df['year'] >= 2015]

# Define the two combinations
combinations = [
    (1, 'Graded/Certified', 'o', '#e74c3c', 3),
    (0, 'Not Graded', 's', '#3498db', 2.5)
]

# Plot each combination as line plot
for auth, label, marker, color, linewidth in combinations:
    subset = df_filtered[df_filtered['authenticity_n'] == auth]
    yearly_counts = subset.groupby('year').size().reset_index(name='count')
    
    # Plot line
    ax.plot(yearly_counts['year'], yearly_counts['count'], marker=marker, linewidth=linewidth,
            label=label, markersize=7, color=color, alpha=0.8)

ax.set_xlabel('Year', fontsize=13)
ax.set_ylabel('Number of Sales', fontsize=13)
ax.set_title('Sales Volume Over Time by Grading Status (2015-2025)', 
             fontsize=15, fontweight='bold', pad=20)
ax.set_xticks(range(2015, 2026, 1))
ax.set_xticklabels([str(year) for year in range(2015, 2026, 1)], rotation=0)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print volume statistics for each segment
print("\n" + "="*80)
print("VOLUME STATISTICS BY GRADING STATUS (2015-2025)")
print("="*80)

for auth, label, _, _, _ in combinations:
    subset = df_filtered[df_filtered['authenticity_n'] == auth]
    yearly_counts = subset.groupby('year').size()
    
    total = len(subset)
    avg_per_year = yearly_counts.mean()
    pct_of_market = (total / len(df_filtered)) * 100
    
    print(f"\n{label}:")
    print(f"  Total sales (2015-2025): {total:,}")
    print(f"  Average per year: {avg_per_year:.0f}")
    print(f"  % of total market: {pct_of_market:.1f}%")
    print(f"  Peak year: {yearly_counts.idxmax()} ({yearly_counts.max():,} sales)")
    print(f"  Low year: {yearly_counts.idxmin()} ({yearly_counts.min():,} sales)")

print("\n" + "="*80)
print(f"Total sales (2015-2025): {len(df_filtered):,}")
print("="*80)

### 🔑 Key Insights:
1. **Strong Growth**: Prices show significant appreciation over time for both graded and non-graded figures
2. **Grading Premium Stable**: The gap between graded and non-graded prices remains consistent throughout the period
3. **Volume Patterns**: Transaction volume shows market maturity with stable demand patterns

## 6. Impact of Grading/Certification

In [ ]:
# Top 10 Most Valuable Figures by Grading Status
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Define the two combinations
combinations = [
    (1, 'Graded/Certified', axes[0], '#e74c3c'),
    (0, 'Not Graded', axes[1], '#3498db')
]

for auth, title, ax, color in combinations:
    # Filter data for this segment
    subset = df[df['authenticity_n'] == auth]
    
    # Get top 10 figures by average price (with at least 5 sales for reliability)
    figure_stats = subset.groupby('figure').agg({
        'selling_price': ['mean', 'count']
    })
    figure_stats.columns = ['avg_price', 'count']
    figure_stats = figure_stats[figure_stats['count'] >= 5]  # At least 5 sales
    top_10 = figure_stats.nlargest(10, 'avg_price')
    
    # Plot
    y_pos = np.arange(len(top_10))
    ax.barh(y_pos, top_10['avg_price'], color=color, alpha=0.8, edgecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_10.index, fontsize=9)
    ax.set_xlabel('Average Selling Price ($)', fontsize=10)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    
    # Add count annotations
    for i, (idx, row) in enumerate(top_10.iterrows()):
        ax.text(row['avg_price'] + 10, i, f"n={int(row['count'])}", 
                va='center', fontsize=8, color='darkgray')

fig.suptitle('Top 10 Most Valuable Figures by Grading Status (min 5 sales)', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Print detailed statistics
print("\n" + "="*80)
print("TOP 10 MOST VALUABLE FIGURES BY GRADING STATUS")
print("="*80)

for auth, title, _, _ in combinations:
    subset = df[df['authenticity_n'] == auth]
    
    figure_stats = subset.groupby('figure').agg({
        'selling_price': ['mean', 'median', 'count']
    })
    figure_stats.columns = ['avg_price', 'median_price', 'count']
    figure_stats = figure_stats[figure_stats['count'] >= 5]
    top_10 = figure_stats.nlargest(10, 'avg_price')
    
    print(f"\n{title}:")
    print("-" * 80)
    for i, (figure, row) in enumerate(top_10.iterrows(), 1):
        print(f"  {i:2d}. {figure[:50]:<50} ${row['avg_price']:7.2f} (n={int(row['count']):3d})")

print("\n" + "="*80)

In [ ]:
# Top 10 Best Selling Figures by Grading Status (by total sales volume)
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Define the two combinations
combinations = [
    (1, 'Graded/Certified', axes[0], '#e74c3c'),
    (0, 'Not Graded', axes[1], '#3498db')
]

for auth, title, ax, color in combinations:
    # Filter data for this segment
    subset = df[df['authenticity_n'] == auth]
    
    # Get top 10 figures by SUMMING the 'sales' column (actual sales volume)
    figure_sales = subset.groupby('figure')['sales'].sum().sort_values(ascending=False).head(10)
    
    # Get average price for each of these top figures
    avg_prices = []
    for figure in figure_sales.index:
        avg_price = subset[subset['figure'] == figure]['selling_price'].mean()
        avg_prices.append(avg_price)
    
    # Create dataframe for plotting
    plot_data = pd.DataFrame({
        'figure': figure_sales.index,
        'total_sales': figure_sales.values,
        'avg_price': avg_prices
    })
    
    # Plot
    y_pos = np.arange(len(plot_data))
    bars = ax.barh(y_pos, plot_data['total_sales'], color=color, alpha=0.8, edgecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(plot_data['figure'], fontsize=9)
    ax.set_xlabel('Total Sales Volume', fontsize=10)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    
    # Add average price annotations
    for i, (idx, row) in enumerate(plot_data.iterrows()):
        ax.text(row['total_sales'] + max(plot_data['total_sales'])*0.02, i, 
                f"${row['avg_price']:.0f}", 
                va='center', fontsize=8, color='darkgreen', fontweight='bold')

fig.suptitle('Top 10 Best Selling Figures by Grading Status (by total sales volume)', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Print detailed statistics
print("\n" + "="*80)
print("TOP 10 BEST SELLING FIGURES BY GRADING STATUS (by total sales volume)")
print("="*80)

for auth, title, _, _ in combinations:
    subset = df[df['authenticity_n'] == auth]
    
    # Sum the 'sales' column for each figure
    figure_sales = subset.groupby('figure')['sales'].sum().sort_values(ascending=False).head(10)
    
    print(f"\n{title}:")
    print("-" * 80)
    for i, (figure, total_sales) in enumerate(figure_sales.items(), 1):
        avg_price = subset[subset['figure'] == figure]['selling_price'].mean()
        num_transactions = len(subset[subset['figure'] == figure])
        print(f"  {i:2d}. {figure[:45]:<45} Sales: {total_sales:6.1f} (avg price: ${avg_price:7.2f}, {num_transactions:3d} transactions)")

print("\n" + "="*80)

### 🔑 Key Insights:
1. **Volume Leaders Consistent**: Same iconic OT characters (Chewbacca, Stormtrooper, Jawa) dominate sales volume across grading status
2. **Total Sales Volume**: Figures show cumulative sales ranging from 200-800 units - represents actual units moved, not transaction count
3. **Price-Volume Relationship**: High-volume figures maintain premium pricing in graded segment ($400-700 avg) - liquidity doesn't mean discount

## 7. Overall Top 10 Figures (All Conditions & Authenticity)

In [ ]:
# Top 10 Most Valuable Figures (Overall - All Conditions & Authenticity)
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# LEFT: Top 10 Most Valuable
ax = axes[0]

# Calculate average price across all conditions
figure_stats = df.groupby('figure').agg({
    'selling_price': ['mean', 'count']
})
figure_stats.columns = ['avg_price', 'count']
figure_stats = figure_stats[figure_stats['count'] >= 5]  # At least 5 sales
top_10_price = figure_stats.nlargest(10, 'avg_price')

y_pos = np.arange(len(top_10_price))
ax.barh(y_pos, top_10_price['avg_price'], color='#9b59b6', alpha=0.8, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(top_10_price.index, fontsize=10)
ax.set_xlabel('Average Selling Price ($)', fontsize=11)
ax.set_title('Top 10 Most Valuable Figures (Overall)', fontsize=13, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add count annotations
for i, (idx, row) in enumerate(top_10_price.iterrows()):
    ax.text(row['avg_price'] + max(top_10_price['avg_price'])*0.02, i,
            f"${row['avg_price']:.0f} (n={int(row['count'])})",
            va='center', fontsize=9, color='darkgray')

# RIGHT: Top 10 Best Selling
ax = axes[1]

# Calculate total sales across all conditions
figure_sales = df.groupby('figure')['sales'].sum().sort_values(ascending=False).head(10)

# Get average price for each
avg_prices = []
for figure in figure_sales.index:
    avg_price = df[df['figure'] == figure]['selling_price'].mean()
    avg_prices.append(avg_price)

plot_data = pd.DataFrame({
    'figure': figure_sales.index,
    'total_sales': figure_sales.values,
    'avg_price': avg_prices
})

y_pos = np.arange(len(plot_data))
ax.barh(y_pos, plot_data['total_sales'], color='#e67e22', alpha=0.8, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(plot_data['figure'], fontsize=10)
ax.set_xlabel('Total Sales Volume', fontsize=11)
ax.set_title('Top 10 Best Selling Figures (Overall)', fontsize=13, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add average price annotations
for i, (idx, row) in enumerate(plot_data.iterrows()):
    ax.text(row['total_sales'] + max(plot_data['total_sales'])*0.02, i,
            f"{row['total_sales']:.0f} units (${row['avg_price']:.0f} avg)",
            va='center', fontsize=9, color='darkgreen', fontweight='bold')

plt.tight_layout()
plt.show()

# Print statistics
print("\n" + "="*80)
print("TOP 10 MOST VALUABLE FIGURES (OVERALL)")
print("="*80)
for i, (figure, row) in enumerate(top_10_price.iterrows(), 1):
    print(f"  {i:2d}. {figure[:50]:<50} ${row['avg_price']:7.2f} (n={int(row['count']):3d})")

print("\n" + "="*80)
print("TOP 10 BEST SELLING FIGURES (OVERALL)")
print("="*80)
for i, (idx, row) in enumerate(plot_data.iterrows(), 1):
    print(f"  {i:2d}. {row['figure'][:50]:<50} {row['total_sales']:6.1f} units (avg: ${row['avg_price']:7.2f})")

print("\n" + "="*80)

### 🔑 Key Insights:
1. **Overall Value Leaders**: Boba Fett and iconic OT heroes dominate - vintage Kenner figures command highest average prices
2. **Volume Champions**: Chewbacca, Stormtrooper, and Jawa lead in total sales - high liquidity items with consistent demand
3. **Price-Volume Balance**: Top sellers maintain strong average prices ($200-400) - market depth exists for popular figures without price dilution

## 8. Grading Impact Analysis

In [ ]:
# Grading/Certification Impact
grading_data = df.groupby('authenticity_n')['selling_price'].agg(['mean', 'median', 'count'])
grading_data.index = ['Not Graded', 'Graded/Certified']
print("Price by Grading Status:")
print(grading_data)

In [ ]:
# Visualization
fig, ax = plt.subplots(figsize=(10, 6))
x = ['Not Graded', 'Graded/Certified']
width = 0.35
x_pos = np.arange(len(x))

grading_data = df.groupby('authenticity_n')['selling_price'].agg(['mean', 'median', 'count'])

ax.bar(x_pos - width/2, grading_data['mean'], width, label='Mean Price', alpha=0.8, color='steelblue')
ax.bar(x_pos + width/2, grading_data['median'], width, label='Median Price', alpha=0.8, color='coral')
ax.set_xlabel('Grading Status', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.set_title('Impact of Grading/Certification on Selling Price', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(x)
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)

# Add count labels
for i, count in enumerate(grading_data['count']):
    ax.text(i, 10, f'n={int(count):,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Add premium annotation
premium_pct = ((grading_data.iloc[1]['mean'] - grading_data.iloc[0]['mean']) / grading_data.iloc[0]['mean']) * 100
ax.text(0.5, grading_data['mean'].max() * 0.9, f'+{premium_pct:.0f}% premium\nfor grading', 
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))

plt.tight_layout()
plt.show()

### 🔑 Key Insights:
1. **Grading Doubles Value**: 92% premium ($344 vs $179) - grading/certification nearly doubles average selling price
2. **Market Split**: 55.5% not graded vs 44.5% graded - grading is mainstream but not dominant, room for growth
3. **ROI Sweet Spot**: Grading cost ($30-$100) justified for figures >$80 ungraded; breakeven at ~$80-$100 base value

## 9. Correlation Analysis

In [ ]:
# Correlation Matrix
numeric_cols = ['authenticity_n', 'selling_price', 'sales', 'year']
correlation = df[numeric_cols].corr()
print("Correlation Matrix:")
print(correlation)

In [ ]:
# Heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix of Numeric Variables')
plt.tight_layout()
plt.show()

### 🔑 Key Insights:
1. **Weak Correlations Overall**: Strongest is only -0.378 (grading vs sales) - these 4 variables explain <20% of price variation
2. **Economics 101**: Price ↔ Sales (-0.225) confirms demand curve - higher prices = lower volume, as expected
3. **Time Stability**: Year ↔ Grading (+0.019) and Year ↔ Sales (+0.047) near zero - market practices and volume remarkably stable over 16 years

## 10. Statistical Testing

In [ ]:
# T-test for grading status
non_graded = df[df['authenticity_n'] == 0]['selling_price']
graded = df[df['authenticity_n'] == 1]['selling_price']
t_stat, p_value = stats.ttest_ind(non_graded, graded)
print(f"T-test (Not Graded vs Graded/Certified):")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.4e}")
print(f"  Result: {'Significant' if p_value < 0.05 else 'Not significant'} difference in prices")

In [ ]:
# Most expensive figure
most_expensive = df.loc[df['selling_price'].idxmax()]
print("KEY INSIGHTS")
print("="*80)
print(f"\nMost expensive sale: {most_expensive['figure']} at ${most_expensive['selling_price']:.2f} in {int(most_expensive['year'])}")

print(f"\nAverage price by grading status:")
for auth, price in df.groupby('authenticity_n')['selling_price'].mean().items():
    auth_label = "Graded/Certified" if auth == 1 else "Not Graded"
    print(f"  - {auth_label}: ${price:.2f}")

print(f"\nTop 5 most valuable character types:")
for i, (char_type, price) in enumerate(df.groupby('character_type')['selling_price'].mean().sort_values(ascending=False).head(5).items(), 1):
    print(f"  {i}. {char_type}: ${price:.2f}")

# Year with highest average price
year_prices = df.groupby('year')['selling_price'].mean()
best_year = year_prices.idxmax()
print(f"\nYear with highest average prices: {int(best_year)} (${year_prices[best_year]:.2f})")

# Correlation insights
corr_year = df['selling_price'].corr(df['year'])
corr_sales = df['selling_price'].corr(df['sales'])
print(f"\nCorrelations:")
print(f"  - Price vs Year: {corr_year:.4f}")
print(f"  - Price vs Sales: {corr_sales:.4f}")